In [ ]:
import random
from mido import Message, MidiFile, MidiTrack
import stilus.midi.utils as utl
import stilus.models as m

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader

In [ ]:
#import a midi file
mid = MidiFile("midi/training/mary.mid")

In [ ]:
# print all events
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        #if msg.type == "note_on":
        print(msg)

In [ ]:
tensor = utl.convert_midi_to_time_series(mid,5,5,8)
print(tensor.shape)
print(tensor[:,0:10])

In [ ]:
conv_net = m.ConvNet_1_0_2()

In [ ]:
conv_net.load_state_dict(torch.load("model_weights/tmpConvNet_1_0_2.pth", map_location=torch.device('cpu')))
conv_net.eval()

In [ ]:
midi_dataset = MidiDataset("training_data.npy")
dataloader = DataLoader(midi_dataset, batch_size=32, shuffle=True)
print(midi_dataset.std)
print(midi_dataset.mean)

In [ ]:
def std_tensor_to_int(pred):
    return ((pred * midi_dataset.std) + midi_dataset.mean).int()

In [ ]:
def int_to_std_tensor(input):
    return ((input - midi_dataset.mean) / midi_dataset.std)

In [ ]:
print(tensor.shape)
std_tensor = int_to_std_tensor(tensor[:,0:0+32].astype("float32"))
print(std_tensor.shape)
tensor_in = torch.unsqueeze(torch.from_numpy(std_tensor),0)
print("tensor_in shape:", tensor_in.shape)
print("tensor_in", tensor[:,0:0+32])
print(tensor_in.dtype)
pred = std_tensor_to_int(conv_net(tensor_in))
print("pred:",  pred)

append = torch.cat((tensor_in[0], torch.t(pred).float()), 1)
print(append)

In [ ]:
#How to create a midi file from scratch
notes = [64, 64+7, 64+12]

outfile = MidiFile()

track = MidiTrack()
outfile.tracks.append(track)

track.append(Message('program_change', program=12))

delta = 32
for i in range(200):
    note = random.choice(notes)
    track.append(Message('note_on', note=note, velocity=100, time=delta))
    track.append(Message('note_off', note=note, velocity=100, time=delta))

outfile.save('midi/new_song.mid')

In [ ]:
outfile.length